In [5]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
import h5py
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import nltk

In [6]:
try:
    import dill as pickle
except ImportError:
    import pickle

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading Caption Generator Model

In [8]:
#model = load(open('./drive/My Drive/Project/theme_Caption_generator_Model.pkl', 'rb'))
model = load(open('./drive/My Drive/Project/urdu/urdu_Caption_generator_Model.pkl', 'rb'))

In [9]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 35, 300)      110700      input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
____________________________________________________________________________________________

In [10]:
#images = './drive/My Drive/Flickr8K_Data/'
images = './drive/My Drive/Project/Theme_dataset/'

In [11]:
#with open("./drive/My Drive/Project/theme_encoded_test_images.pkl", "rb") as encoded_pickle:
#with open("./drive/My Drive/Project/urdu/urdu_encoded_test_images.pkl", "rb") as encoded_pickle:
with open("./drive/My Drive/Project/urdu/urdu_encoded_test_images.pkl", "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

In [12]:
# Loading wordtoix
#wordtoix = load(open("./drive/My Drive/Project/theme_wordtoix.pkl", "rb"))
wordtoix = load(open("./drive/My Drive/Project/urdu/urdu_wordtoix.pkl", "rb"))
print('Wordtoix Length = %d' % len(wordtoix))

Wordtoix Length = 368


In [13]:
ixtoword = {}
for word, i in wordtoix.items():
    ixtoword[i] = word

In [14]:
Directory_path = "./drive/My Drive/Project/Theme_dataset/"

In [15]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [16]:
max_length = 35

In [17]:
z =0
results=[]
while (z<len(encoding_test)):
  pic = list(encoding_test.keys())[z]
  image = encoding_test[pic].reshape((1,2048))
  #image = encoding_test[pic].reshape((1,4096))
  results.append(greedySearch(image))
  x=plt.imread(Directory_path+pic)
  plt.axis('off')
  plt.imshow(x)
  plt.show()
  print(greedySearch(image))
  z+=1

Output hidden; open in https://colab.research.google.com to view.

# Bleu Score Calculation

In [18]:
filepath = './drive/My Drive/Project/urdu/test_captions.txt'
ground_captions=[]
with open(filepath, encoding='utf-8') as fp:
    names = fp.readline()
    while names:
        names=names.replace("\n","")
        ground_captions.append(names)
        names = fp.readline()

In [19]:
z=0
results=[]
Bleuscore=[]
while (z<len(encoding_test)):
  pic = list(encoding_test.keys())[z]
  image = encoding_test[pic].reshape((1,2048))
  #image = encoding_test[pic].reshape((1,4096))
  results.append(greedySearch(image))

  score = nltk.translate.bleu_score.sentence_bleu([ground_captions[z*5],ground_captions[(z*5)+1],ground_captions[(z*5)+2],ground_captions[(z*5)+3],ground_captions[(z*5)+4]], results[z])
  print("image",z,score)
  Bleuscore.append(score)
  z+=1

print("Bleu Score of model is:", (sum(Bleuscore)*100)/len(encoding_test))

image 0 0.6370997536905525
image 1 0.5163785701734271
image 2 0.8206825711567206
image 3 0.03990656068619026
image 4 0.07206716452215016
image 5 0.5010254781249093
image 6 0.8524016761333578
image 7 0.1319210615440637
image 8 0.7306085739678484
image 9 0.3659627171776776
image 10 0.34768265977767104
image 11 0.2811622078391865
image 12 0.579117562367794
image 13 0.46659769599819473
image 14 0.4774376125905535
image 15 0.30807517149745295
image 16 0.2583338814978223
image 17 0.5112539860568288
image 18 0.499675332896607
image 19 0.7118786505502754
image 20 0.44329254637494486
image 21 0.37548560991324814
image 22 0.7817953032840942
image 23 0.5923717260998315
image 24 0.6301058114224722
image 25 0.6866815260073993
image 26 0.5986221031572082
image 27 0.6763264743579778
image 28 0.5640652755139506
image 29 0.7809843077320021
image 30 0.2544696343787984
image 31 0.5074180027066014
image 32 0.2596008172511666
image 33 0.40299434371189324
image 34 0.871797993236827
image 35 0.27237304392385

In [20]:
z =0
results=[]
pic = list(encoding_test.keys())[z]

In [21]:
pic

'3174417550_d2e6100278.jpg'

In [22]:
  image = encoding_test[pic]

In [23]:
image.shape

(2048,)